In [2]:
import json
import pandas
from preprocessing import extract_runes_data
from ctk_scrollable_dropdown import *
from pulp import *
from tabulate import tabulate
import sys

properties = [
    "hp",
    "hp%",
    "atk",
    "atk%",
    "def",
    "def%",
    "spd",
    "crt",
    "crd",
    "res",
    "acc",
]
weights = [
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
]


def optimize(runes, weights=weights, stats={}, sets={}):
    model = LpProblem(sense=LpMaximize)
    items = pulp.LpVariable.dicts("rune", indices=runes.index, cat=LpBinary).values()

    model += sum(
        sum(
            weights * props * item
            for props, item in zip(runes[properties].values, items)
        )
    )

    # slot constraints
    for i in range(1, 7):
        model += (
            sum(item for slot, item in zip(runes.slot_no.values, items) if slot == i)
            <= 1
        )

    # stat constraints
    for stat, count in stats.items():
        model += sum(stat * item for stat, item in zip(runes[stat], items)) >= count

    # set constraints
    for set, count in sets.items():
        model += (
            sum(set_flag * item for set_flag, item in zip(runes[set], items)) >= count
        )

    return dict(
        status=LpStatus[model.solve(PULP_CBC_CMD(msg=True))],
        objective=value(model.objective),
        solution={int(str(item)[5:]): item.value() for item in items if item.value()},
    )

In [3]:
data = json.load(open("runes.json", "r", encoding="utf-8"))
runes = extract_runes_data(pandas.DataFrame(data["runes"]))
runes = pandas.get_dummies(runes, columns=["set_id"])
runes.head()

,occupied_id,slot_no,hp,hp%,atk,atk%,def,def%,spd,crt,...,set_id_16,set_id_17,set_id_18,set_id_19,set_id_20,set_id_21,set_id_22,set_id_23,set_id_24,set_id_25
rune_id,,,,,,,,,,,,,,,,,,,,,
45682013155,0,1,0.0,0.0,118.0,0.0,0.0,0.0,5.0,6.0,...,0,0,0,0,0,0,0,0,0,0
45760452800,0,1,350.0,5.0,22.0,0.0,0.0,0.0,4.0,4.0,...,0,0,0,0,0,0,0,0,0,0
45769181302,0,1,309.0,6.0,118.0,0.0,0.0,0.0,16.0,5.0,...,0,0,0,0,0,0,0,0,0,0
45774686197,0,1,353.0,0.0,22.0,0.0,0.0,0.0,0.0,6.0,...,0,0,0,0,0,0,0,0,0,0
45802705341,0,1,0.0,5.0,22.0,0.0,0.0,0.0,0.0,6.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
sets = [col for col in runes if "set_id" in col]

In [23]:
import tkinter as tk
from tkinter import filedialog, messagebox
from customtkinter import *


def load_json():
    file_path = filedialog.askopenfilename(filetypes=[("JSON files", "*.json")])
    if file_path:
        with open(file_path, "r", encoding="utf-8") as file:

            data = json.load(file)
            data = pandas.DataFrame(data["runes"])
            runes = extract_runes_data(data)
            dummies = pandas.get_dummies(runes, columns=["set_id"])
            result = optimize(dummies)
            status = result["status"].lower()
            print(f"Solution is {status}")
            if status != "infeasible":
                print(
                    tabulate(
                        runes.loc[result["solution"].keys()],
                        headers="keys",
                        tablefmt="psql",
                    )
                )


# Create the main application window
app = CTk()
app.geometry("2200x1000")
app.title("Summoners War Optimizer")
app.minsize(1200, 800)
set_appearance_mode("dark")
set_default_color_theme("blue")


frame = CTkFrame(app)
frame.pack(pady=20, padx=20, fill="both", expand=True)

load_json_btn = CTkButton(frame, command=load_json, height=100)
load_json_btn.grid(row=0, column=0, padx=20, pady=20, sticky="ew", columnspan=2)

frame.grid_columnconfigure((0, 1), weight=1)
frame.grid_columnconfigure((2, 3), weight=10)
frame.grid_rowconfigure((1), weight=10)

settings_frame = CTkFrame(frame)
settings_frame.grid(row=1, column=0, padx=20, pady=20, sticky="ewns")

CTkLabel(settings_frame, text="Monster").grid(row=0, column=0, padx=20, pady=20)
CTkEntry(settings_frame, justify="center", text_color="orange").grid(
    row=0, column=1, padx=20, pady=20
)

CTkLabel(settings_frame, text="Time limit").grid(row=1, column=0, padx=20, pady=20)
CTkEntry(settings_frame, justify="center", text_color="orange").grid(
    row=1, column=1, padx=20, pady=20
)

# CTkLabel(settings_frame, text="Logger").grid(row=2, column=0, padx=20, pady=20)
CTkCheckBox(
    settings_frame, text="Logger", variable=(logging := BooleanVar(app, False))
).grid(row=2, column=0, padx=20, pady=20)

# CTkLabel(settings_frame, text="Fresh").grid(row=3, column=0, padx=20, pady=20)
CTkCheckBox(
    settings_frame, text="Fresh", variable=(fresh := BooleanVar(app, False))
).grid(row=2, column=1, padx=20, pady=20)


sets_frame = CTkFrame(frame)
sets_frame.grid(row=2, column=0, padx=20, pady=20, sticky="ew")

weights_frame = CTkFrame(frame)
weights_frame.grid(row=1, column=1, padx=20, pady=20, sticky="ew", rowspan=2)

for i, p in enumerate(properties):
    plabel = CTkLabel(weights_frame, text=p)
    plabel.grid(row=i, column=0, padx=20, pady=20)
    pentry = CTkEntry(
        weights_frame, placeholder_text=1, justify="center", text_color="orange"
    )
    pentry.grid(row=i, column=1, padx=20, pady=20)


def set_picker():
    entry = CTkEntry(sets_frame)
    entry.pack(fill="x", padx=20, pady=20)

    def insert_method(e):
        entry.delete(0, "end")
        entry.insert(0, e)

    CTkScrollableDropdown(
        entry,
        values=sets,
        command=lambda e: insert_method(e),
        autocomplete=True,
        button_color="transparent",
    )
    return entry


drops = [set_picker() for i in range(3)]

text = CTkTextbox(frame, wrap="word")
text.grid(row=0, column=2, padx=20, pady=20, sticky="ewns", columnspan=2, rowspan=3)
text.configure(text_color="orange", font=("monospace", 10))


def write(string):
    text.configure(state="normal")
    text.insert("end", string)
    text.configure(state="disabled")


sys.stdout.write = write

# Start the application
app.mainloop()
print("end")

invalid command name "139964905231168<lambda>"
    while executing
"139964905231168<lambda>"
    ("after" script)
invalid command name "139964906281216destroy_popup"
    while executing
"139964906281216destroy_popup"
    ("after" script)
invalid command name "139964906287744destroy_popup"
    while executing
"139964906287744destroy_popup"
    ("after" script)
invalid command name "139964907946560destroy_popup"
    while executing
"139964907946560destroy_popup"
    ("after" script)


TclError: invalid command name ".!ctkframe.!ctktextbox.!text"